In [ ]:
import gensim
import re
from konlpy.tag import Twitter

MINIMUM_SENTENCE_LENGTH = 3

input_file_name="/mnt/data/test_data/wiki_ko.txt"
output_file_name="/mnt/data/test_data/korean_sentences_for_word2vec.txt"

In [ ]:
ex_hangul = re.compile('[^ ㄱ-ㅣ가-힣|.]+') # 한글과 띄어쓰기를 제외한 모든 글자
ex_split_sentence = ['',' ']
def map_hangul(str):  #한글만 추출하는 메소드
    str = str.replace('_',' ')
    str = ex_hangul.sub('',str)
    str = str.replace('  ',' ')
    return str

def split_sentence(str): #마침표로 문장을 나누고, 빈 문장을 제거하는 메소드
    sentence_arr = str.split('.')
    return list(filter(lambda x: x not in ex_split_sentence, sentence_arr))
    

In [ ]:
#문장 전처리
import time
start = int(time.time())

twitter_nlp = Twitter()
output_file = open(output_file_name, "a")
index = 0

with open(input_file_name, "r") as f:
    while True:
        try:
            line = f.readline()
            if not line:
                break

            index += 1
            if index % 100000 == 0:
                print('step:',index)

            tmp_sentences = split_sentence(map_hangul(line)) #한글 추출, 라인 split
            for sentence in tmp_sentences:
                tmp_noun_arr = twitter_nlp.nouns(sentence) #명사 추출
                if len(tmp_noun_arr) > MINIMUM_SENTENCE_LENGTH:
                    output_file.write(' '.join(tmp_noun_arr)+'\n')
        except Exception as e:
            print(e)

output_file.close()
end = int(time.time())
print(end-start)

In [1]:
import gensim

#word2vec 학습
def iter_file(file_name):
    with open(file_name, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break;
            yield line.replace('\n','').split(' ')
    
it = iter_file('/mnt/data/test_data/korean_sentences_for_word2vec.txt')

model = gensim.models.Word2Vec(size=300, window=5, min_count=50, workers=2)
model.build_vocab(it)
model.save('dict_data/kor_w2v_model')

In [2]:
it = iter_file('/mnt/data/test_data/korean_sentences_for_word2vec.txt')
model.train(it)
model.save('dict_data/kor_w2v_model_trained')

In [7]:
print(model.wv.similarity('미국', '일본'))
print(model.wv.most_similar(positive=['문재인', '박근혜'], negative=['노무현']))
print(model.wv.most_similar(positive=['테란', '마린'], negative=['프로토스']))

0.29413325883
[('김무성', 0.6947234869003296), ('안철수', 0.6834571957588196), ('원내대표', 0.6675081253051758), ('이정희', 0.6563081741333008), ('김한길', 0.6384867429733276), ('심상정', 0.6365790367126465), ('추미애', 0.6296842098236084), ('김종인', 0.6270934343338013), ('강기갑', 0.621680736541748), ('한명숙', 0.6175681352615356)]
[('파이어뱃', 0.4452653229236603), ('벌처', 0.42615044116973877), ('인베이더', 0.4211110472679138), ('장경환', 0.4169806241989136), ('배너티', 0.41055530309677124), ('어썰트', 0.40961235761642456), ('선호산', 0.40626388788223267), ('칠리치', 0.4044717848300934), ('트데드', 0.40154731273651123), ('드랍쉽', 0.40134942531585693)]


In [ ]:
# new_model = gensim.models.Word2Vec.load('dict_data/w2v_model_wiki_kor')